In [8]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [9]:
# Set dataset paths
dataset_path = r"C:\Users\madas\OneDrive\Desktop\model3\session1\dataset.csv"
image_dir = r"C:\Users\madas\OneDrive\Desktop\model3\session1"

# Load CSV file
df = pd.read_csv(dataset_path)

# Image settings
IMG_SIZE = (128, 128)
data, labels = [], []

# Read images and labels
for _, row in df.iterrows():
    img_path = os.path.join(image_dir, row['filename'])
    label = row['label']

    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    if img is not None:
        img = cv2.resize(img, IMG_SIZE)
        data.append(img)
        labels.append(label)

# Convert to numpy arrays and normalize
data = np.array(data) / 255.0  
data = data.reshape(-1, 128, 128, 1)  # Add channel dimension
labels = to_categorical(labels)  # One-hot encode labels

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [10]:
datagen = ImageDataGenerator(
    rotation_range=10,  # Reduce rotation
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,  # Small zoom
    fill_mode="nearest"
)


In [15]:
from tensorflow.keras.layers import Activation

# Define Improved CNN Model
model = Sequential([
    Conv2D(32, (5,5), input_shape=(128,128,1)),  # Larger Kernel
    BatchNormalization(),
    Activation('swish'),  # Swish Activation
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), padding='same'),
    BatchNormalization(),
    Activation('swish'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), padding='same'),
    BatchNormalization(),
    Activation('swish'),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), padding='same'),  # Extra Conv Layer
    BatchNormalization(),
    Activation('swish'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='swish'),
    Dropout(0.4),
    Dense(labels.shape[1], activation='softmax')  
])

# Compile with lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0002), loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
datagen = ImageDataGenerator(
    rotation_range=20,  
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],  # Adjust brightness
    fill_mode="nearest"
)


In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_test, y_test),
    epochs=100,  
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
150/150 [==============================] - 180s 1s/step - loss: 1.6591 - accuracy: 0.2217 - val_loss: 1.6613 - val_accuracy: 0.1975
Epoch 2/100
150/150 [==============================] - 190s 1s/step - loss: 1.6930 - accuracy: 0.2240 - val_loss: 30.3628 - val_accuracy: 0.1933
Epoch 3/100
150/150 [==============================] - 203s 1s/step - loss: 1.6445 - accuracy: 0.2452 - val_loss: 269.9903 - val_accuracy: 0.1775
Epoch 4/100
150/150 [==============================] - 174s 1s/step - loss: 1.6712 - accuracy: 0.2454 - val_loss: 988.3759 - val_accuracy: 0.2275
Epoch 5/100
150/150 [==============================] - 174s 1s/step - loss: 1.7041 - accuracy: 0.2450 - val_loss: 2830.4446 - val_accuracy: 0.1617
Epoch 6/100
150/150 [==============================] - 167s 1s/step - loss: 1.6547 - accuracy: 0.2438 - val_loss: 8161.1943 - val_accuracy: 0.1617
Epoch 7/100
150/150 [==============================] - 150s 998ms/step - loss: 1.6738 - accuracy: 0.2483 - val_loss: 3007.159

In [14]:
# Save the trained model
model.save("optimized_model.h5")

# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


C:\Users\madas\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


38/38 [==============================] - 6s 156ms/step - loss: 0.6112 - accuracy: 0.7400
Test Accuracy: 74.00%
